## 発表スライドの計算

In [1]:
using LinearAlgebra

A = [
    1 3 5 5
    1/3 1 3 5
    1/5 1/3 1 2
    1/5 1/5 1/2 1
]

_, n = size(A)

λ = eigvals(A)[4]

4.131451830581148 + 0.0im

In [2]:
w = real.(eigvecs(A)[1:n,4])
w = w / sum(w)

4-element Vector{Float64}:
 0.5475207090938252
 0.2739308638789637
 0.10872386651043692
 0.0698245605167741

In [3]:
(A - real(λ) * Matrix{Float64}(I, n, n)) * w

4-element Vector{Float64}:
 -8.881784197001252e-16
  2.220446049250313e-16
 -2.220446049250313e-16
 -1.6653345369377348e-16

In [4]:
using JuMP
using HiGHS
using IntervalArithmetic

function upperAppoximation(A)
    ε = 1e-8

    model = Model(HiGHS.Optimizer)
    set_silent(model)

    try
        @variable(model, wᴸ[i=1:n] ≥ ε); @variable(model, wᵁ[i=1:n] ≥ ε)

        for i = 1:n
            wᵢᴸ = wᴸ[i]; wᵢᵁ = wᵁ[i]
            @constraint(model, wᵢᵁ ≥ wᵢᴸ)

            ∑wⱼᴸ = sum(map(j -> wᴸ[j], filter(j -> i != j, 1:n)))
            @constraint(model, ∑wⱼᴸ + wᵢᵁ ≤ 1)
            ∑wⱼᵁ = sum(map(j -> wᵁ[j], filter(j -> i != j, 1:n)))
            @constraint(model, ∑wⱼᵁ + wᵢᴸ ≥ 1)

            for j = 1:n
                if i == j continue end

                aᵢⱼ = A[i,j]
                wⱼᴸ = wᴸ[j]; wⱼᵁ = wᵁ[j]

                @constraint(model, aᵢⱼ * wⱼᵁ ≥ wᵢᴸ)
                @constraint(model, aᵢⱼ * wⱼᴸ ≤ wᵢᵁ)
            end
        end

        @objective(model, Min, -sum(wᴸ) + sum(wᵁ))
        optimize!(model)

        return map(i -> value(wᴸ[i])..value(wᵁ[i]), 1:n)
    finally
        empty!(model)
    end
end

upperAppoximation (generic function with 1 method)

In [5]:
upperAppoximation(A)

4-element Vector{Interval{Float64}}:
 [0.601851, 0.601852]
 [0.200617, 0.231482]
 [0.0771604, 0.120371]
 [0.0462962, 0.120371]